In [ ]:
# with fixed train, val dataset folders

import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim


data_dir = 'h:\\Yehmh\\BCI\\2020_08_01_seg_images_labeled'
batch_size  = 32
num_classes = len(os.listdir(data_dir))
num_epochs = 25
learning_rate = 0.001

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


# Datasets and dataloaders
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#Load a pretrained model
model = models.resnet50(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
model = model.to(device)

#Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)
    
    val_loss = val_running_loss / len(val_loader.dataset)
    val_acc = val_corrects.double() / len(val_loader.dataset)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

print("Training complete.")

In [3]:
# split datasets to train and validate

import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch
import torch.nn as nn
import torch.optim as optim


data_dir = 'h:\\Yehmh\\BCI\\2020_08_01_seg_images_labeled'
batch_size  = 32
num_classes = len(os.listdir(data_dir))
num_epochs = 100
learning_rate = 0.001
train_split_ratio = 0.8

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Load the full dataset
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# Split dataset into training and validation sets
train_size = int(train_split_ratio * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#Load a pretrained model
model = models.resnet50(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
model = model.to(device)

#Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)
    
    val_loss = val_running_loss / len(val_loader.dataset)
    val_acc = val_corrects.double() / len(val_loader.dataset)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

print("Training complete.")

Epoch [1/100], Loss: 1.4525144247863193
Validation Loss: 9.630434799194337, Validation Accuracy: 0.40714285714285714
Epoch [2/100], Loss: 0.9856421577224184
Validation Loss: 1.556088525908334, Validation Accuracy: 0.5928571428571429
Epoch [3/100], Loss: 0.8627856742971889
Validation Loss: 0.9842129571097238, Validation Accuracy: 0.6571428571428571
Epoch [4/100], Loss: 0.7644282425327403
Validation Loss: 0.8131490656307765, Validation Accuracy: 0.6642857142857143
Epoch [5/100], Loss: 0.6104701307988466
Validation Loss: 2.2774658679962156, Validation Accuracy: 0.4857142857142857
Epoch [6/100], Loss: 0.6927473894997602
Validation Loss: 1.0488749538149154, Validation Accuracy: 0.6642857142857143
Epoch [7/100], Loss: 0.5654471758452095
Validation Loss: 1.5749140296663557, Validation Accuracy: 0.5785714285714285
Epoch [8/100], Loss: 0.39614030932823674
Validation Loss: 1.2806351406233651, Validation Accuracy: 0.6857142857142857
Epoch [9/100], Loss: 0.2909110182063584
Validation Loss: 2.20362

In [ ]:
# Plot the confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

model.eval()
val_running_loss = 0.0
val_corrects = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        val_corrects += torch.sum(preds == labels.data)

val_loss = val_running_loss / len(val_loader.dataset)
val_acc = val_corrects.double() / len(val_loader.dataset)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")